In [1]:
import pandas as pd
import numpy as np
import requests
from citipy import citipy
from api_keys import weather_api_key

In [2]:
csv_path='data/Alaska_parks.csv'

parks_loc = pd.read_csv(csv_path, low_memory=False)
parks_loc

,Unnamed: 0,park_code,national_park,latitude,longitude
0,14,DENA,Denali National Park,63.33,-150.50
1,18,GAAR,Gates of the Arctic National Park,67.78,-153.30
2,20,GLBA,Glacier Bay National Park,58.50,-137.00
3,32,KATM,Katmai National Park,58.50,-155.00
4,33,KEFJ,Kenai Fjords National Park,59.92,-149.65
5,34,KOVA,Kobuk Valley National Park,67.55,-159.28
6,35,LACL,Lake Clark National Park,60.97,-153.42
7,52,WRST,Wrangell - St Elias National Park,61.00,-142.00


In [3]:
parks_loc_df=parks_loc.drop(columns='Unnamed: 0')
parks_loc_df

,park_code,national_park,latitude,longitude
0,DENA,Denali National Park,63.33,-150.50
1,GAAR,Gates of the Arctic National Park,67.78,-153.30
2,GLBA,Glacier Bay National Park,58.50,-137.00
3,KATM,Katmai National Park,58.50,-155.00
4,KEFJ,Kenai Fjords National Park,59.92,-149.65
5,KOVA,Kobuk Valley National Park,67.55,-159.28
6,LACL,Lake Clark National Park,60.97,-153.42
7,WRST,Wrangell - St Elias National Park,61.00,-142.00


In [4]:
parks_loc_df.dtypes

park_code         object
national_park     object
latitude         float64
longitude        float64
dtype: object

In [9]:
lat_lng_df=parks_loc_df[['latitude','longitude']].astype(int)
lat_lng_df


,latitude,longitude
0,63,-150
1,67,-153
2,58,-137
3,58,-155
4,59,-149
5,67,-159
6,60,-153
7,61,-142


In [13]:
# make lat long list 
# lat_lngs = []
cities = []
for idx, lat_lng in lat_lng_df.iterrows():
    city = citipy.nearest_city(lat_lng['latitude'], lat_lng['longitude']).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
cities

['wasilla',
 'college',
 'sitka',
 'kodiak',
 'sterling',
 'barrow',
 'homer',
 'haines junction']

In [15]:
# api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}
url='https://api.openweathermap.org/data/2.5/weather?'
api_key=f'&appid={weather_api_key}'
units= '&units=imperial'
citys=[]
lat=[]
lon=[]
maxtemp=[]
mintemp=[]
hum=[]
cloud=[]
wspeed=[]
country=[]
date=[]
city_cnt=0
for city in cities:#replace for short print test in range: cities[1:5]
    city_cnt=city_cnt+1
    city_ref=f'q={city}'
    response = requests.get(url+city_ref+units+api_key).json()
    try:
        lat.append(response['coord']['lat'])
        lon.append(response['coord']['lon'])
        citys.append(response['name'])
        maxtemp.append(response['main']['temp_max'])
        mintemp.append(response['main']['temp_min'])
        hum.append(response['main']['humidity'])
        cloud.append(response['clouds']['all'])
        wspeed.append(response['wind']['speed'])
#      
    except KeyError:
        print("No City Found Skipping --------------------")
    print(f"Processing Record:{city_cnt}  of Set:{1} |{city}")

Processing Record:1  of Set:1 |wasilla
Processing Record:2  of Set:1 |college
Processing Record:3  of Set:1 |sitka
Processing Record:4  of Set:1 |kodiak
Processing Record:5  of Set:1 |sterling
Processing Record:6  of Set:1 |barrow
Processing Record:7  of Set:1 |homer
Processing Record:8  of Set:1 |haines junction


In [17]:
weather_dic={
    'City':citys,
    'Lat':lat,
    'Lng':lon,
    'Max Temp':maxtemp,
    'Min Temp':mintemp,
    'Humidity':hum,
    'Cloudiness':cloud,
    'Wind Speed':wspeed,
}

weather_data=pd.DataFrame(weather_dic)

weather_data['Lat'] = weather_data['Lat'].map('{:,.2f}'.format).astype(float)
weather_data['Lng'] = weather_data['Lng'].map('{:,.2f}'.format).astype(float)
weather_data

,City,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Date
0,Wasilla,61.58,-149.44,12.20,10.99,52,1,3.44,1615666539
1,College,64.86,-147.80,-5.01,-18.40,84,1,2.80,1615666562
2,Sitka,57.05,-135.33,33.80,33.80,55,40,14.97,1615666283
3,Saint Paul Harbor,57.79,-152.41,30.20,30.20,37,1,17.27,1615666563
4,Sterling,39.01,-77.43,57.99,55.00,18,75,5.91,1615666414
5,Barrow,71.29,-156.79,-9.40,-9.40,70,1,16.11,1615666564
6,Homer,59.64,-151.55,17.60,14.00,49,1,8.05,1615666564
7,Haines Junction,60.75,-137.51,3.20,3.20,47,83,14.97,1615666565
